In [1]:
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from performance_tests.cfsm_incrementer import CFSMIncrementer
from performance_tests.resources.cfsm import cfsm as initial_cfsm, initial_index

incrementer = CFSMIncrementer()
index = initial_index
points_number = 5
randomize_times = 10

/home/diego/.pyenv/versions/3.6.5/envs/bisimulation/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
def size_of_cfsm(cfsm):
    states = len(cfsm.states)
    transitions = sum(len(transitions) for transitions in cfsm.transitions_by_source_id.values())
    return states * transitions

In [3]:
def calculate_bisimulation_time_for(cfsm, randomize_times):
    times = []
    for i in range(randomize_times):
        print(f'  {i+1} randomize')
        randomized_cfsm = incrementer.randomize(cfsm)

        start_time = time.time()
        result = cfsm.calculate_bisimulation_with(randomized_cfsm)
        end_time = time.time()
        print(f'{end_time - start_time} seconds')

        times.append(end_time - start_time)

    return np.mean(times)

In [4]:
def get_bisimulation_metrics_using(cfsm, points_number, randomize_times, increment_strategy):
    metrics = []
    for i in range(points_number):
        if i != 0:
            cfsm = increment_strategy(cfsm)

        size = size_of_cfsm(cfsm)
        print(f'Calculating: {i+1} metric, size: {size}')
        avg_time = calculate_bisimulation_time_for(cfsm, randomize_times)
        metric = {'size': size, 'avg_time': avg_time}
        print(metric)
        print('')
        metrics.append(metric)

    return metrics

---
## Simple increment strategy

In [5]:
def simple_increment_strategy(cfsm):
    global simple_increment_index
    global simple_increment_size
    new_cfsm, new_index = incrementer.increment(cfsm, simple_increment_size, simple_increment_index)
    cfsm_array.append(new_cfsm)
    simple_increment_index = new_index
    return new_cfsm

In [ ]:
cfsm_array = []
simple_increment_index = initial_index
simple_increment_size = 2
simple_metrics = get_bisimulation_metrics_using(
    initial_cfsm,
    points_number=points_number,
    randomize_times=randomize_times,
    increment_strategy=simple_increment_strategy
)

simple_df = pd.DataFrame(simple_metrics)
simple_df.to_csv('performance_tests/results/simple_increment.csv', index=False)

In [7]:
def increment_splitting_transitions_strategy(cfsm):
    global splitting_transitions_index
    global splitting_transitions_size
    cfsm, new_index = incrementer.increment_splitting_transitions(cfsm, splitting_transitions_size, splitting_transitions_index)
    splitting_transitions_index = new_index
    return cfsm

In [ ]:
splitting_transitions_index = initial_index
splitting_transitions_size = 2
splitting_transitions_metrics = get_bisimulation_metrics_using(
    initial_cfsm,
    points_number=points_number,
    randomize_times=randomize_times,
    increment_strategy=increment_splitting_transitions_strategy
)

splitting_transitions_df = pd.DataFrame(splitting_transitions_metrics)
splitting_transitions_df.to_csv('performance_tests/results/increment_splitting_transitions.csv', index=False)

---
## Increment Splitting transitions in new states strategy

In [9]:
def increment_splitting_transitions_in_new_states_strategy(cfsm):
    global splitting_transitions_in_new_states_index
    global splitting_transitions_in_new_states_size
    cfsm, new_index = incrementer.increment_splitting_transitions(cfsm, splitting_transitions_in_new_states_size, splitting_transitions_in_new_states_index)
    splitting_transitions_in_new_states_index = new_index
    return cfsm

In [ ]:
splitting_transitions_in_new_states_index = initial_index
splitting_transitions_in_new_states_size = 2
splitting_transitions_metrics = get_bisimulation_metrics_using(
    initial_cfsm,
    points_number=points_number,
    randomize_times=randomize_times,
    increment_strategy=increment_splitting_transitions_in_new_states_strategy
)

splitting_transitions_in_new_states_df = pd.DataFrame(splitting_transitions_metrics)
splitting_transitions_in_new_states_df.to_csv('performance_tests/results/increment_splitting_transitions_in_new_states.csv', index=False)